In [1]:
from nltk.corpus import stopwords
#from nltk.cluster.util import cosine_distance
import numpy as np
import math
import networkx as nx   # FOR(PAGE RANK)
import re
import string

In [2]:
def readarticle(filename):
    file=open(filename,'r')
    filedata=file.read()
    article=filedata.split(".")
    sentences=[]
    for sentence in article:
        sentences.append(sentence.strip().replace("[^a-zA-Z]", " ").split(" "))
    return sentences

In [3]:
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from nltk.corpus import stopwords
import re
def readarticle_lema(filename):
    file=open(filename,'r',encoding="utf-8")
    file_text=file.read()
    
    org_text = file_text
    org_text=re.sub("[()]",".",org_text)
    orginal = org_text.split(".")
    if len(orginal[-1])<5:
        orginal.pop(-1)
        
    
    file_text=re.sub("[()]",".",file_text)
    file_text=re.sub("[^a-zA-Z0-9.]"," ",file_text)
    file_text=re.sub('["]',' ',file_text)
    sents= file_text.split(".")
    if len(sents[-1])<5:
        sents.pop(-1)
    words=list()
    wordnet_lemmatizer = WordNetLemmatizer()
    for sent in sents:
        sent=sent.strip()
        lema_words=[wordnet_lemmatizer.lemmatize(word.lower(), pos="v") for word in sent.split()]
        words.append([word for word in lema_words if word not in list(stopwords.words('english'))])
    return orginal,words

def readarticle_stma(file_text):
    #file=open(filename,'r',encoding="utf-8")
    #file_text=file.read()
    
    org_text = file_text
    org_text=re.sub("[()]",".",org_text)
    orginal = org_text.split(".")
    if len(orginal[-1])<5:
        orginal.pop(-1)
    
    file_text=re.sub("[()]",".",file_text)
    file_text=re.sub("[^a-zA-Z0-9.]"," ",file_text)
    file_text=re.sub('["]',' ',file_text)
    sents= file_text.split(".")
    if len(sents[-1])<5:
        sents.pop(-1)
    words=list()
    lancaster=LancasterStemmer()
    for sent in sents:
        sent=sent.strip()
        stema_words=[lancaster.stem(word.lower()) for word in sent.split()]
        words.append([word for word in stema_words if word not in list(stopwords.words('english'))])
    return orginal,words

In [4]:
def get_uniquewords(sentences):
    stop_words=stopwords.words('english')
    unque=set()
    for sentence in sentences:
        for word in sentence:
            if word not in stop_words:
                unque.add(word)
    return sorted(list(unque))

# EXTRACTIVE METHODS

## 1 . TextRank (Page rank)

In [5]:
def cosine_similarity(A,B):
    denom =np.linalg.norm(A)*np.linalg.norm(B)
    if denom==0:
        return 0.7
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
    return cosine_similarity(vector1, vector2)
    #return 1 - cosine_distance(vector1, vector2)

In [6]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        sent1=[i.lower() for i in sentences[idx1]]
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            sent2=[i.lower() for i in sentences[idx2]]
            similarity_matrix[idx1][idx2] = sentence_similarity(sent1, sent2, stop_words)

    return similarity_matrix

In [7]:
def TextRank_summary(file_name,ABSTRACT_SIZE=0.3):
    stop_words=stopwords.words('english')
    summary=[]
    act_sentences,sentences=readarticle_stma(file_name)
    similarityMatrix=build_similarity_matrix(sentences,stop_words)
    
    
    #ranking the sentences using Page Rank
    sentence_similarity_graph = nx.from_numpy_array(similarityMatrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    sorted_scores=list(sorted(scores.items(),key= lambda item:item[1],reverse=True))[:max(1,math.floor(len(act_sentences)*ABSTRACT_SIZE))]
    
    return ".".join([act_sentences[i[0]] for i in sorted(sorted_scores[:])])


In [8]:
TextRank_summary("text.txt")

'In an ramraj attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. This program also included developer-focused AI school that provided a bunch of assets to help build AI skills'

## 2 . Luhn's method (feature based )

In [9]:
def top_words(sentences):
    record = {}
    common_words =  stopwords.words('english')  #load_common_words()
    for sentence in sentences:
        words = sentence.split()
        for word in words:  #sentences is already a list of words so no need to split again. .
            w = word.strip('.!?,()\n').lower()
            record[w]= record.get(w,0)+1

    for word in record.keys():
        if word in common_words:
            record[word] = -1     
    occur = [key for key in record.keys()]
    occur.sort(reverse=True, key=lambda x: record[x])
    return set(occur[: len(occur) // 10 ])

In [10]:
def calculate_score(sentence, metric):
    words = sentence.split()
    imp_words, total_words, begin_unimp, end, begin = [0]*5
    for word in words:
        w = word.strip('.!?,();').lower()
        end += 1
        if w in metric:
            imp_words += 1
            begin = total_words
            end = 0
        total_words += 1
    unimportant = total_words - begin - end
    if(unimportant != 0):
        return float(imp_words**2) / float(unimportant)
    return 0.0

In [11]:
def Luhn_summary(file_name,ABSTRACT_SIZE=0.3):
    actual,sentences = readarticle_stma(file_name)
    sentences = [" ".join(sentence) for sentence in sentences] #to make words list to sentence
    metric = top_words(sentences)
    scores = {}
    for i,sentence in enumerate(sentences):
        scores[i]=calculate_score(sentence, metric)
    

    #sorting according to the ranking
    sorted_scores=list(sorted(scores.items(),key= lambda item:item[1],reverse=True))[:max(1,math.floor(len(actual)*ABSTRACT_SIZE))]
    
    return ".".join([actual[i[0]] for i in sorted(sorted_scores)])


In [12]:
Luhn_summary("text.txt")

'In an ramraj attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning'

## 3. LSA ( cross )

In [13]:
def modified_tfidf(sentences , unique_words,modified=True):
    tf_idf= np.zeros((len(sentences),len(unique_words)))
    tot_frequency=dict()
    i=0
    # frequency matrix or TF values
    for sentence in sentences:
        for word in sentence :
            if word in unique_words:
                j =unique_words.index(word)
                freq = tf_idf[i][j]
                if freq==0 :
                    tot_frequency[word]=tot_frequency.get(word,0)+1
                tf_idf[i][j]=freq+1
        i=i+1
    #print(tot_frequency)
    #binary=tf_idf
    # calculating IDF values for all the unique values
    x,y = tf_idf.shape
    idf={}
    for i in tot_frequency.keys():
        idf[i]=math.log(x/tot_frequency[i])
    #print(idf)
    # calculating tf_idf values 
    for i in range(x):
        for j in range(y):
            tf_idf[i][j] = tf_idf[i][j]*idf[unique_words[j]]
    if modified:
        # modified Tf_IDF approch making the less than average values to zero to remove noise
        sent_avg = np.mean(tf_idf,axis=1)
        #print("average= ",sent_avg)
        res=[]
        for i in range(x):
            res.append(list(np.greater(tf_idf[i],sent_avg[i]).astype("int")))
        #print(np.count_nonzero(tf_idf==0) , np.count_nonzero((res*tf_idf)==0))
        return res*tf_idf
    else:
        return tf_idf

In [16]:
def LSA_summary(filename,ABSTRACT_SIZE=0.3):
    orginal,sentences = readarticle_stma(filename)
    uniqueWords=get_uniquewords(sentences)
    tf_idf_vectors=modified_tfidf(sentences,uniqueWords)
    #print(len(sentences), len(uniqueWords), tf_idf_vectors.shape)
    U,s,V = np.linalg.svd(np.transpose(tf_idf_vectors))
    V_avg=np.mean(V,axis=1)
    #print("avg= " ,V_avg)
    
    # redusing the noices again 
    res=[]
    for i in range(len(V_avg)):
        res.append(list(np.greater(V[i],V_avg[i]).astype("int")))
    V= V*res
    
    # geting the sentence length values
    Lengths = np.sum(V,axis=0)
    scores = {i:j for i,j in enumerate(Lengths)}

    #sorting according to the ranking
    sorted_scores=list(sorted(scores.items(),key= lambda item:item[1],reverse=True))[:max(1,math.floor(len(orginal)*ABSTRACT_SIZE))]
    #print(sorted_scores)
    # Selecting the top sentences
    
    return ".".join([orginal[i[0]] for i in sorted(sorted_scores)])

In [17]:
LSA_summary("text.txt")


' The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry'

## 4 . Fuzzi logic

In [18]:
from fuzzyLogic.summerize import fuzzy_summary

In [19]:
fuzzy_summary("text.txt")

before feature calculation
feature 1 done
feature 2 done
feature 3 done
feature 4 done
feature 5 done
feature 6 done


ZeroDivisionError: division by zero

In [3]:
import nltk

In [4]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lokanadh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lokanadh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

## 5. Sentence Embidings (ref)

In [1]:
### DATA SET AMAZON REVIEWS DATA SET (OPTIONAL)
### PRE TRIENDED 50 DIMENSSIONS EMBIDINGS(GLOVE)
### k-means clustering and selection based on the distance to the center of the cluster.

In [20]:
import numpy as np
import re
import string

In [5]:
def loadEmbeddingMatrix(EMBEDDING_FILE):
    embeddings_index = dict()
    #Transfer the embedding weights i dictionary by iterating through every line of the file.
    f = open(EMBEDDING_FILE,'r',encoding='utf-8')
    for line in f:
        #split up line into an indexed array
        values = line.split()
        #first index is word
        word = values[0]
        #store the rest of the values in the array as a new array
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs #50 dimensions
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))

    return embeddings_index #, embedding_matrix

## Loading 'glove' words
emb_index= loadEmbeddingMatrix('C:/Users/lokanadh/Desktop/Anaconda/text summerization/glove.6B.50d.txt')

Loaded 400000 word vectors.


In [8]:
def get_sent_embedding(wordlist):
    """
    This function calculates the embedding of each sentence in the review. Checks if the sentence being passed is a valid one, 
    removing the punctuation and emojis etc.
    """
    sent_emb = []
    for i in wordlist:
        i = i.lower()
        try :
            res=list(emb_index[i])
            print(i,sep="    ",end="")
        except:
            res=list(emb_index['unknown'])
            print('""'+i+'""',end="    ")
        print(res,end="\n\n")
        sent_emb.append(res)

    #calculating the mean 
    
    #for empty senetnces
    if len(sent_emb)<1:
        return np.zeros(50)
    sent_emb=np.mean(sent_emb,axis=0)
    return np.array(sent_emb)

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

def Embeding_summery(file_name):
    actual , sentences =readarticle_lema(file_name)
    emb_sents=[get_sent_embedding(sent) for sent in sentences]
    sentences=[" ".join(sent) for sent in sentences]
    n_clusters = int(np.ceil(len(emb_sents)**0.5))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(emb_sents)
    avg = []
    closest = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        #print("IDX is: ", idx)
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,emb_sents)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summary = ' '.join([actual[closest[idx]] for idx in ordering])
    return summary

Embeding_summery("text.txt")

""ramraj""    [0.89855, 0.30093, 0.38384, -0.07748, 1.2406, 0.6338, -0.49759, 0.59377, -0.16398, -0.079284, 0.6614, -0.17841, 0.064431, 0.15498, 0.63783, -0.12535, -0.045814, 0.084162, -0.84272, 0.25469, -0.53641, 0.058337, 0.53229, 0.60801, 0.41529, -1.2192, -1.1077, -0.29251, 0.50284, 0.65703, 2.2331, -1.2356, 0.18461, -1.1709, 0.56209, 0.3741, 0.24536, -0.21032, -0.35088, 0.20336, 0.098822, -0.15596, 0.088795, 0.17909, 0.21729, -0.50994, -0.48693, -0.07791, 0.55245, -0.62789]

attempt[0.5323, -0.70044, 0.37422, -0.31529, 0.012028, 0.50933, -0.23546, 0.72083, -0.1387, 0.5066, -0.38464, 0.12579, -0.68992, 0.38425, -0.1367, 0.083163, 0.62144, -1.0586, -0.75507, -0.0057474, 0.53155, 0.097558, -0.41206, -0.40328, 0.31674, -2.2257, -0.043076, -0.83749, 0.93522, -0.21468, 2.4972, 0.2487, -1.7374, -0.48367, -0.051459, 0.2463, -0.25872, -0.20222, -0.43626, -0.4603, -0.11058, -0.019738, 0.089743, -0.35528, 0.15794, -0.24601, 0.30675, 0.02953, 0.63595, -0.59635]

build[1.2426, 0.56913, 1.0025,

' The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning  This will require more collaborations and training and working with AI  Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry and the jobs of tomorrow will require a different skillset'

## 6. Own method (based on paper)

In [24]:
import numpy as np
import math

In [25]:
def SVD_elimenation_clusterWise(tf_idf_vectors,indx,elimiate=False):
    U,s,V = np.linalg.svd(np.transpose(tf_idf_vectors))
    V_avg=np.mean(V,axis=1)
    #print("avg= " ,V_avg)
    
    # redusing the noices again 
    res=[]
    for i in range(len(V_avg)):
        res.append(list(np.greater(V[i],V_avg[i]).astype("int")))
    if elimiate:
        V= V*res
    
    # geting the sentence length values
    Lengths = np.sum(V,axis=0)
    return {i:j for i,j in zip(indx,Lengths)}

In [26]:
def get_abstract_ratio(counts,ABSTRACT_SIZE):
    f_c=[]
    c=[]
    for _,i in counts.items():
        f_c.append(math.floor(i*ABSTRACT_SIZE))
        c.append(float(i)*ABSTRACT_SIZE)
    return f_c,c

In [27]:
def remove_bottom_30(selected,scores):
    scores = sorted(scores.items(),key=lambda item:item[1])[:round(len(selected)*0.40)]
    for i in scores:
        selected[i[0]]=-1
    return selected

In [28]:
def get_sent_embedding_tfidf(wordlist,tf_idf,unique_words):
    """
    This function calculates the embedding of each sentence in the review. Checks if the sentence being passed is a valid one, 
    removing the punctuation and emojis etc.
    """
    sent_emb = []
    for i in wordlist:
        i = i.lower()
        try :
            res=list(emb_index[i])
        except:
            res=list(emb_index['unknown'])
            
        sent_emb.append(np.array(res)*tf_idf[unique_words.index(i)])

    # multiplying with the tf_idf values
    #calculating the mean 
    sent_emb=np.mean(sent_emb,axis=0)
    return np.array(sent_emb)

In [29]:
def select_sentences_high(selected,scores,ratios):
    '''
    need to the cluster preference order based on the average sentence value.
    '''
    sele=0
    for i in range(len(ratios)):
        j=0
        #print("cluster []",i)
        sorted_ist = sorted(scores[i].items(),key=lambda item:item[1],reverse=True)
        while j<ratios[i]:
            top = sorted_ist.pop(0)
            index = top[0]
            if selected[index]==0:
                sele=sele+1
                selected[index]=1
                j=j+1
    return selected,sele
        
        

In [30]:
def select_sentences_low(selected , scores ,get_ratio):
    min_cover = [float(math.ceil(i))-i for i in get_ratio]
    min_index =min_cover.index(min(min_cover))
    j=0
    sorted_list=sorted(scores[min_index].items(),key=lambda item:item[1],reverse=True)
    while j<1:
        top = sorted_list.pop(0)
        index = top[0]
        if selected[index]==0:
            selected[index]=1
            j=j+1
    return selected,min_index

In [71]:
def OWN_summary(filename,ABSTRACT_SIZE=0.3):
    actual,sentences = readarticle_lema(filename)
    uniqueWords=get_uniquewords(sentences)
    tf_idf = modified_tfidf(sentences,uniqueWords,modified=True)
    print("loaded text and converted to matrix: done")
    # sentence wise normalizationof tf_idf values
    res=[]
    for i in tf_idf:
        denom=max(i)-min(i)
        if denom==0:
            res.append(i)
        else:
            res.append((i-min(i))/(max(i)-min(i)))
    res = np.array(res)+1
    print("done1")
    # getting the sentence embedding 
    emb_sents = [get_sent_embedding_tfidf(sentences[i],res[i],uniqueWords) for i in range(len(sentences))]
    
    # clustering the sentences
    print("started clustering  :",end=" ")
    n_clusters = int(np.ceil(len(emb_sents)**0.5))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(emb_sents)
    print("done")
    
    print("applying SVD alogo :",end=" ")
    # implementation of the SvD method to give attention to the required sentences in each clusters.
    tf_idf_cluster=np.array(tf_idf)
    scores_cluster = {}
    counts={}
    for i in range(n_clusters):
        idx = list(np.where(kmeans.labels_ == i)[0])
        scores_cluster[i]=SVD_elimenation_clusterWise(tf_idf_cluster[idx],idx,elimiate=True)
        counts[i]=len(idx)
    # implementation of the SvD method to give attention on global level.
    scores_global = SVD_elimenation_clusterWise(tf_idf_cluster,[i for i in range(len(actual))])
    print("done")
    
    selected=[0 for i in range(len(actual))]
    
    # the bellow process eliminates bottom 30% of text before the selection step.
    selected = remove_bottom_30(selected,scores_global)
    #print(selected)
    print("removed bottom 40 percentel using globel level : done")
    # get top ABSTRACT_SIZE percent sente form each cluster
    
    print("constructing summary from top :",end=" ")
    
    get_ratio_f,get_ratio= get_abstract_ratio(counts,ABSTRACT_SIZE)
    
    to_be_selected=math.floor(len(actual)*ABSTRACT_SIZE)
    selected,cur_selected = select_sentences_high(selected,scores_cluster,get_ratio_f)
    print("done")
    while(cur_selected<to_be_selected):
        print("....calling function to rescue")
        selected , cl= select_sentences_low(selected , scores_cluster ,get_ratio)
        get_ratio[cl]=math.ceil(get_ratio[cl])
        cur_selected=cur_selected+1
    print("\n\n",selected,"\n")
    
    summary=[]
    for i in range(len(selected)):
        if selected[i]==1:
            summary.append(actual[i])
    return ".".join(summary)

In [32]:
dic = OWN_summary("text.txt")
print(dic)

loaded text and converted to matrix: done
started clustering  : done
applying SVD alogo : done
removed bottom 40 percentel using globel level : done
constructing summary from top : done
....calling function to rescue


 [-1, 0, 1, -1, 0, -1, 0, 1, -1, 0, 1, -1] 

 As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow. The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well


## TEST AREA

In [35]:
from bs4 import BeautifulSoup
# from urllib.request import urlopen
from urllib.request import urlopen
def get_text(url):
	page = urlopen(url)
	soup = BeautifulSoup(page)
	fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
	return fetched_text

In [54]:
text = get_text('https://en.wikipedia.org/wiki/Mahesh_Babu')
print("done")
act,sentences=readarticle_stma(text)
uniq = get_uniquewords(sentences)

done


In [75]:
res=np.array([[1.,2.3,4.32,3.4]])
np.mean([[0,0,0,0,0]],axis=0)

array([0., 0., 0., 0., 0.])

In [77]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# EVALUATION

In [ ]:
!pip install rouge-score

In [1]:
import text_summerizer as TS

In [6]:
import pandas as pd
df = pd.read_csv("cleaned_CNN.csv")
df

,articles,summaries
0,it is official american . president barack o...,syrian official obama climbed to the top of t...
1,cnn usain bolt rounded off the world cham...,usain bolt wins third gold of world championsh...
2,"kansas city, missouri cnn the general ser...",the employee in agencys kansas city office is ...
3,los angeles cnn a medical doctor in vanco...,new a canadian doctor says she was part of a ...
4,cnn police arrested another teen thursday...,another arrest made in gang rape outside calif...
...,...,...
195,cnn arsenal came back from two goals down...,arsenal came back from two goals down to claim...
196,cnn getting hired or promoted in todays c...,"every company has an emotional motivation, or ..."
197,when harry returned home the morning after the...,residents of tacloban returning to home to che...
198,sydney cnn indian prime minister narendra...,"australia returns lost art treasures to india,..."


In [72]:
raw_text_list = df['articles'].values
line= '____________________________________________________________________________________________________'
i=1
pred_summ= list()
article =list()
for art in list(raw_text_list):
    try:
        summ = TS.Embeding_summery(art,ABSTRACT_SIZE=0.05)
    except:
        summ = "###"
    pred_summ.append(summ)
    print(i,": done")
    i=i+1

1 : done
2 : done
3 : done
4 : done
5 : done
6 : done
7 : done
8 : done
9 : done
10 : done
11 : done
12 : done
13 : done
14 : done
15 : done
16 : done
17 : done
18 : done
19 : done
20 : done
21 : done
22 : done
23 : done
24 : done
25 : done
26 : done
27 : done
28 : done
29 : done
30 : done
31 : done
32 : done
33 : done
34 : done
35 : done
36 : done
37 : done
38 : done
39 : done
40 : done
41 : done
42 : done
43 : done
44 : done
45 : done
46 : done
47 : done
48 : done
49 : done
50 : done
51 : done
52 : done
53 : done
54 : done
55 : done
56 : done
57 : done
58 : done
59 : done
60 : done
61 : done
62 : done
63 : done
64 : done
65 : done
66 : done
67 : done
68 : done
69 : done
70 : done
71 : done
72 : done
73 : done
74 : done
75 : done
76 : done
77 : done
78 : done
79 : done
80 : done
81 : done
82 : done
83 : done
84 : done
85 : done
86 : done
87 : done
88 : done
89 : done
90 : done
91 : done
92 : done
93 : done
94 : done
95 : done
96 : done
97 : done
98 : done
99 : done
100 : done
101 : do

In [73]:
Embeding_df=pd.DataFrame({'actual':df['summaries'],'predicted':pred_summ})
Embeding_df.to_csv("res\Embeding_results.csv",index=False)

In [74]:
from rouge_score import rouge_scorer
scores=list()
scorer = rouge_scorer.RougeScorer(['rouge1','rougeL'], use_stemmer=True)
for i,j in zip(Embeding_df['actual'].values,Embeding_df['predicted'].values):
    if len(j)>10:
        scores.append(scorer.score(i, j ))

In [75]:
p=[]
r=[]
f=[]
pp=[]
rr=[]
ff=[]
for i in scores:
    p.append(i['rouge1'].precision)
    r.append(i['rouge1'].recall)
    f.append(i['rouge1'].fmeasure)
    pp.append(i['rougeL'].precision)
    rr.append(i['rougeL'].recall)
    ff.append(i['rougeL'].fmeasure)   

In [76]:
import numpy as np
print("rouge1 = ",np.mean(p),np.mean(r),np.mean(f))
print("rougeL = ",np.mean(pp),np.mean(rr),np.mean(ff))

rouge1 =  nan nan nan
rougeL =  nan nan nan


C:\Users\lokanadh\anaconda3\envs\project\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\lokanadh\anaconda3\envs\project\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
